In [1]:
import numpy as np

# The Grid World Environment
states = list(range(16)) # 0-15
terminal_states = [0, 15]
actions = ['n', 'e', 's', 'w'] # North, East, South, West
gamma = 1.0 # Undiscounted as per the slide
reward = -1.0 # Constant reward for all transitions

# Initialize a value function table
V = np.zeros(len(states))
print("Initial Value Function (V_0):")
print(V.reshape(4, 4)) 

Initial Value Function (V_0):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [2]:
#Define the uniform random policy Pi(a|s) = 0.25 for all actions
policy = np.full((len(states), len(actions)), 0.25) 
print("\n--- 'PREDICTION' PROBLEM SETUP ---")
print("INPUT: A full MDP model and a policy (uniform random).")
print("GOAL: Find the value function, V_pi, for this policy.") 


--- 'PREDICTION' PROBLEM SETUP ---
INPUT: A full MDP model and a policy (uniform random).
GOAL: Find the value function, V_pi, for this policy.


In [3]:
print("\n--- 'CONTROL' PROBLEM SETUP ---")
print("INPUT: A full MDP model.")
print("GOAL: Find the optimal value function, V*, and the optimal policy, pi*.") 


--- 'CONTROL' PROBLEM SETUP ---
INPUT: A full MDP model.
GOAL: Find the optimal value function, V*, and the optimal policy, pi*.


In [4]:
def policy_evaluation(policy,V,gamma,reward,threshold=1e-6):
    while True:
        delta = 0 
        V_old = V.copy() 

        for s in range(len(states)):
            if s in terminal_states:
                continue 
            new_v = 0
            #apply bellman expectation equation
            for a_idx,action_prob in enumerate(policy[s]):
                next_s = get_next_state(s,actions[a_idx])

                #new_v = sum(Pi(a|s) * (R(s,a,s') + gamma * V(s'))) 
                new_v += action_prob *(reward +gamma *V[next_s])
            V[s] = new_v 
        
        #Check for convergence 
        delta = max(delta,np.max(np.abs(V-V_old)))
        if delta < threshold: 
            break 
    return V  

In [5]:
def get_next_state(s, action):
    """
    Given a state and action, returns the next state.
    Handles grid boundaries.
    """
    row, col = divmod(s, 4)
    if action == 'n':
        row = max(row - 1, 0)
    elif action == 's':
        row = min(row + 1, 3)
    elif action == 'w':
        col = max(col - 1, 0)
    elif action == 'e':
        col = min(col + 1, 3)
    return row * 4 + col 

In [6]:
print("\n--- Running Iterative Policy Evaluation ---")
# Start with V_0 = all zeros
V_eval = np.zeros(len(states))
# Use the uniform random policy
uniform_policy = np.full((len(states), len(actions)), 0.25)

V_pi = policy_evaluation(uniform_policy, V_eval, gamma, reward)
print("Converged Value Function for the Random Policy (V_pi):")
print(np.round(V_pi.reshape(4, 4), 1))
print("(Compare with k=inf on slide 11)") 


--- Running Iterative Policy Evaluation ---
Converged Value Function for the Random Policy (V_pi):
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
(Compare with k=inf on slide 11)


In [7]:
def policy_improvement(V, gamma, reward):
    """
    Improves a policy by acting greedily with respect to a value function.
    """
    new_policy = np.zeros_like(policy) # Start with a new policy table
    
    for s in range(len(states)):
        if s in terminal_states:
            continue
            
        # Calculate q-value for each action
        q_values = []
        for a_idx, action in enumerate(actions):
            next_s = get_next_state(s, action)
            q = reward + gamma * V[next_s]
            q_values.append(q)
            
        # The new policy is to deterministically choose the best action
        best_action_idx = np.argmax(q_values)
        new_policy[s, best_action_idx] = 1.0 # Set probability to 1 for the best action
        
    return new_policy 

In [8]:
# Start with the uniform random policy
current_policy = np.full((len(states), len(actions)), 0.25)
V_control = np.zeros(len(states))

while True:
    # 1. Evaluate the current policy
    V_control = policy_evaluation(current_policy, V_control, gamma, reward)
    
    # 2. Improve the policy by acting greedily
    new_policy = policy_improvement(V_control, gamma, reward)
    
    # 3. Check for convergence
    if np.array_equal(new_policy, current_policy):
        print("Policy has converged!")
        break
        
    current_policy = new_policy

print("\nOptimal Value Function (V*):")
print(np.round(V_control.reshape(4, 4), 1))
print("\nOptimal Policy (pi*):")
# Convert numeric policy to arrows for display
arrows = ['↑', '→', '↓', '←'] 
optimal_actions = np.argmax(current_policy, axis=1)
print(np.array([arrows[i] for i in optimal_actions]).reshape(4,4))

Policy has converged!

Optimal Value Function (V*):
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Optimal Policy (pi*):
[['↑' '←' '←' '↓']
 ['↑' '↑' '↑' '↓']
 ['↑' '↑' '→' '↓']
 ['↑' '→' '→' '↑']]
